In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Log in the huggingface here

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model, ViTFeatureExtractor, ViTModel, T5Tokenizer, T5ForConditionalGeneration, BartTokenizer, BartForConditionalGeneration, BertModel
from moviepy.editor import VideoFileClip
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, EncoderDecoderModel, BertConfig, EncoderDecoderConfig
import numpy as np
from PIL import Image
import librosa
from sklearn.decomposition import PCA

Change the dir to the on in your drive, note, there should be one CMU_MOSI folder contains video in same dir

In [4]:
os.chdir('/content/drive/MyDrive/USC_Course/Multimodal_Probabilistic_Learning_of_Human_CSCI535/Group_Project')

### Extract audio and frames data from video

Only need run once, I recommend to just generate 100 nearly samples, and manually stop it

In [15]:
def extract_audio_and_frames(video_path, audio_output_path, frame_output_dir):
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_output_path)

    if not os.path.exists(frame_output_dir):
        os.makedirs(frame_output_dir)

    for i, frame in enumerate(video.iter_frames()):
        frame_image = Image.fromarray(frame)
        frame_image.save(f"{frame_output_dir}/frame_{i:05d}.png")

def process_all_videos(video_folder, audio_folder, frames_folder):
    # Ensure the output directories exist
    os.makedirs(audio_folder, exist_ok=True)
    os.makedirs(frames_folder, exist_ok=True)

    # Loop through all mp4 files in the video folder
    for video_file in os.listdir(video_folder):
        if video_file.endswith(".mp4"):
            video_path = os.path.join(video_folder, video_file)
            base_filename = os.path.splitext(video_file)[0]

            audio_output_path = os.path.join(audio_folder, f"{base_filename}.wav")
            frame_output_dir = os.path.join(frames_folder, base_filename)

            extract_audio_and_frames(video_path, audio_output_path, frame_output_dir)
            print(f"Processed {video_file}")

In [18]:
video_folder = "./CMU_MOSI"
audio_folder = "./100_wav"
frames_folder = "./100_video_frames"


In [ ]:
if not os.path.exists(audio_folder) and not os.path.exists(frames_folder):
  process_all_videos(video_folder, audio_folder, frames_folder)

MoviePy - Writing audio in ./100_wav/VCslbP0mgZI_16.wav


MoviePy - Done.


  warnings.warn("Warning: in file %s, "%(self.filename)+



Processed VCslbP0mgZI_16.mp4
MoviePy - Writing audio in ./100_wav/BXuRRbG0Ugk_12.wav


MoviePy - Done.


...

Processed cW1FSBF59ik_16.mp4
MoviePy - Writing audio in ./100_wav/I5y0__X72p0_5.wav


MoviePy - Done.


  warnings.warn("Warning: in file %s, "%(self.filename)+



Processed I5y0__X72p0_5.mp4
MoviePy - Writing audio in ./100_wav/2iD-tVS8NPw_20.wav


MoviePy - Done.


### Extract features

In [8]:
audio_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
audio_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

# Load ViT
video_feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
video_model = ViTModel.from_pretrained("google/vit-base-patch16-224")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

  warnings.warn(



config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def extract_audio_features(audio_path, processor, model):
    import librosa
    waveform, sr = librosa.load(audio_path, sr=16000)
    inputs = processor(waveform, return_tensors="pt", sampling_rate=sr)
    with torch.no_grad():
        hidden_states = model(**inputs).last_hidden_state
    return hidden_states.mean(dim=1)

def extract_video_features(frame_dir, feature_extractor, model):
    frames = [Image.open(os.path.join(frame_dir, frame)) for frame in sorted(os.listdir(frame_dir)) if frame.endswith('.png')]
    inputs = feature_extractor(images=frames, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def process_features(audio_folder, frames_folder):
    audio_features_dict = {}
    video_features_dict = {}
    i=0

    for audio_file in sorted(os.listdir(audio_folder)):
      if i==0:
        if audio_file.endswith(".wav"):
            base_filename = os.path.splitext(audio_file)[0]
            audio_path = os.path.join(audio_folder, audio_file)
            frame_dir = os.path.join(frames_folder, base_filename)
            print(audio_path, frame_dir)
            if os.path.exists(frame_dir):
                print(f"Processing {audio_file} and corresponding frames...")
                audio_features = extract_audio_features(audio_path, audio_processor, audio_model)
                video_features = extract_video_features(frame_dir, video_feature_extractor, video_model)

                audio_features_dict[base_filename] = audio_features
                video_features_dict[base_filename] = video_features
            else:
                print(f"No corresponding frame directory found for {audio_file}")
      i =i+1

    return audio_features_dict, video_features_dict

In [13]:
audio_folder = "./wav"
frames_folder = "./video_frames"

# Process all features
audio_features_dict, video_features_dict = process_features(audio_folder, frames_folder)